In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)  #no limit to the number of columns shown
import sctoolbox.utils as utils
import sctoolbox.tools as tools
import sctoolbox.plotting as pl
import sctoolbox.tools.receptor_ligand as rl
from sctoolbox import settings
settings.settings_from_config("config.yaml", key="11")

__Use `help(<function_name>)` for information on the parameters. E.g. `help(rl.download_db)`__

------------------

In [ ]:
# Path to receptor ligand database
db_path = '/mnt/flatfiles/receptor_ligand/zebrafish_rl_db_cellchat.txt'

# Comput interactions
cluster_col = "clustering"
normalize = 1000 # Correct clusters to given size.
gene_col = None # Column that holds gene name /id. Set to None to use index
overwrite = False # Overwrite existing interaction table

# Plotting

dpi = 100

## Violin
violin_min_perc = 0 # Minimum percentage of cells in a cluster that express the respective gene.
violin_out_suffix = "violin" # Suffix of output file
violin_figsize = (5, 30) # Figure size

## Network
net_min_perc = 0 # Minimum percentage of cells in a cluster that express the respective gene.
net_interaction_score = 0 # Interaction score must be above this threshold for the interaction to be counted in the graph.
net_interaction_perc = 0 # Select interaction scores above or equal to the given percentile. Will overwrite parameter interaction_score.
net_out_suffix = "network" # Suffix of output file
net_title = None # Plot title
net_color_min = 0 # Min value for color range
net_color_max = None # Max value for color range
net_restrict_to = [] # Only show given clusters provided in list.
net_show_count = True # Show the interaction count
# cyclone specific parameters
net_directional = False # Show the direction of the interactions (ligand -> receptor)
net_sector_size_is_cluster_size = False # Sector width is based on number of cells
net_show_genes = True # Show the top receptor & ligand genes for each cluster
net_gene_amount = 5 # The amount of top receptor & ligand genes that are shown for each cluster

## Connection plot
con_restrict_to = None # Restrict plot to given cluster names.
con_figsize = (5, 10) # Figure size
con_out_suffix="connectionPlot" # Suffix of output file
con_title = None # Plot title
con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 0 & receptor_percent >= 50 & ligand_percent >= 50" # Conditions to filter the interaction table
con_lw_multiplier = 2 # Linewidth multiplier
con_wspace = 0.4 # Width between plots

## Filter for interaction table
tab_min_perc = 0
tab_interaction_score = 0
tab_interaction_perc = 0
tab_out_suffix = "interaction_table"
group_a = None
group_b = None

In [ ]:
def filter_cyclone(adata, interactions_table, min_count):
    """
    Filter interactions between clusters that have less than min_count
    """
    adata_tmp = adata.copy()
    
    cluster_col_list = ["receptor_cluster", "ligand_cluster"]

    interactions_directed = pd.DataFrame(
        interactions_table[cluster_col_list].value_counts(),
        columns=["count"]
    )

    interactions_directed.reset_index(inplace=True)

    # Adds the number of interactions for each pair of receptor and ligand
    # if they should be displayed non directional.

    # e.g.: ligand A interacts with receptor B five times
    #       and ligand B interacts with receptor A ten times
    #       therefore A and B interact fifteen times non directional
    interactions_undirected = interactions_directed.copy()

    # First, make sure each edge is represented in the same way, by sorting the two ends
    interactions_undirected[cluster_col_list] = np.sort(
        interactions_undirected[cluster_col_list].values,
        axis=1
    )

    # Then, you can group by the two columns and sum the connections
    interactions_undirected = interactions_undirected.groupby(cluster_col_list)["count"].sum().reset_index()
    # <<< OpenAI GPT-4 supported

    interactions = interactions_undirected
    filtered = interactions[interactions['count'] > min_count]
    # remove intra-connections
    filtered = filtered[filtered['receptor_cluster'] != filtered['ligand_cluster']]
    filtered = filtered[['receptor_cluster', 'ligand_cluster']].values
    filtered = [(a, b) for a, b in filtered]
    
    to_keep = []
    for index, row in interactions_table.iterrows():
        if (row['receptor_cluster'], row['ligand_cluster']) in filtered or (row['ligand_cluster'], row['receptor_cluster']) in filtered:
            to_keep.append(index)
    
    adata_tmp.uns["receptor-ligand"]["interactions"] = interactions_table.iloc[to_keep]
    
    return adata_tmp

# Endocardium sub-clusters

## Load data
The dataset must be clustered for a receptor-ligand analysis.

In [ ]:
adata_endo = utils.adata.load_h5ad("anndata_5_Endocardium_sub_clusters.h5ad")
display(adata_endo)

In [ ]:
# Visualize cells in UMAP
sc.pl.umap(adata_endo, color = ['sub_clusters'], title = [], legend_loc = 'right margin')

## Download receptor-ligand database
Provide a path to a table with receptor-ligand interactions. Will be stored in the adata object (`adata.uns['receptor-ligand]['database']`).

In [ ]:
rl.download_db(adata=adata_endo,
               db_path='/mnt/flatfiles/receptor_ligand/zebrafish_rl_db_cellchat.txt',
               ligand_column='ligand',
               receptor_column='receptor',
               inplace=True,
               overwrite=True)

## Compute interactions
Create a table of receptor-ligand interactions between given groups. This table is used for plotting. Will be stored in the adata object (`adata.uns['receptor-ligand']['interactions']`)

### Clusters

In [ ]:
rl.calculate_interaction_table(adata=adata_endo,
                               cluster_column="sub_clusters",
                               gene_index=None,
                               normalize=1000,
                               inplace=True,
                               overwrite=False)

# Plotting

## Violin
Show the distribution of interaction scores for all group combinations.

Scores `> 0` can be interpreted as receptor-ligand interactions enriched for group combination.  
Scores `< 0` can be interpreted as receptor-ligand interactions depleted for group combination.

In [ ]:
rl.interaction_violin_plot(adata_endo,
                           min_perc=violin_min_perc,
                           save=f"rl_{violin_out_suffix}_endo.pdf",
                           figsize=violin_figsize,
                           dpi=dpi)

## Network
Show a network graph of number of interactions between groups.

In [ ]:
rl.hairball(adata_endo,
        min_perc=net_min_perc,
        interaction_score=net_interaction_score,
        interaction_perc=net_interaction_perc,
        save=f"rl_{net_out_suffix}_endo.pdf",
        title=net_title,
        color_min=net_color_min,
        color_max=net_color_max,
        restrict_to=net_restrict_to,
        show_count=net_show_count
       )

## Receptor-ligand connections
Show a detailed view on receptor-ligand pairs and their strength between groups.

- __receptor-/ ligand score__: Gene enriched (`> 0`) or depleted (`< 0`) for specific group.
- __receptor-/ ligand percent__: Percent of cells in a group expressing gene.
- __interaction score__: receptor-ligand pair enriched (`> 0`) or depleted (`< 0`) between groups.

In [ ]:
con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 0 & receptor_percent >= 50 & ligand_percent >= 50" # Conditions to filter the interaction table

rl.connectionPlot(adata=adata_endo, 
                  restrict_to=con_restrict_to,
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}_endo_subclusters.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")

In [ ]:
interaction_table = rl.get_interactions(adata_endo,
                                        min_perc = tab_min_perc,
                                        interaction_score = tab_interaction_score,
                                        interaction_perc = tab_interaction_perc,
                                        save = f"rl_{tab_out_suffix}_endo_subclusters.tsv")
interaction_table.reset_index(drop=True, inplace=True)
interaction_table

In [ ]:
_ = rl.cyclone(adata=adata_endo,
               min_perc=net_min_perc,
               interaction_score=net_interaction_score,
               interaction_perc=net_interaction_perc,
               #save=f"rl_cyclone_{net_out_suffix}_endo_subclusters.pdf",
               color_min=net_color_min,
               color_max=net_color_max,
               title=net_title,
               directional=net_directional,
               sector_size_is_cluster_size=net_sector_size_is_cluster_size,
               show_genes=net_show_genes,
               gene_amount=net_gene_amount,
               dpi=300, figsize=(10, 10), sector_text_size=10
)

In [ ]:
# exclude intra-connections and connections with count < 500
endo_filtered = filter_cyclone(adata_endo, interaction_table, min_count=500)

In [ ]:
_ = rl.cyclone(adata=endo_filtered,
               min_perc=net_min_perc,
               interaction_score=net_interaction_score,
               interaction_perc=net_interaction_perc,
               save=f"rl_cyclone_{net_out_suffix}_endo_subclusters.pdf",
               color_min=500,
               color_max=net_color_max,
               title=net_title,
               directional=net_directional,
               sector_size_is_cluster_size=net_sector_size_is_cluster_size,
               show_genes=net_show_genes,
               gene_amount=net_gene_amount,
               dpi=300, figsize=(10, 10), sector_text_size=10
)

In [ ]:
_ = rl.cyclone(adata=endo_filtered,
               min_perc=net_min_perc,
               interaction_score=net_interaction_score,
               interaction_perc=net_interaction_perc,
               save=f"rl_cyclone_{net_out_suffix}_endo_subclusters_nogene.pdf",
               color_min=500,
               color_max=net_color_max,
               title=net_title,
               directional=net_directional,
               sector_size_is_cluster_size=net_sector_size_is_cluster_size,
               show_genes=False,
               gene_amount=net_gene_amount,
               dpi=300, figsize=(10, 10), sector_text_size=10, colormap='Blues'
)

In [ ]:
interaction_table = rl.get_interactions(endo_filtered,
                                        min_perc = tab_min_perc,
                                        interaction_score = tab_interaction_score,
                                        interaction_perc = tab_interaction_perc,
                                        save = f"rl_{tab_out_suffix}_endo_subclusters_filtered.tsv")
interaction_table.reset_index(drop=True, inplace=True)
interaction_table

# Myocardium

## Load data
The dataset must be clustered for a receptor-ligand analysis.

In [ ]:
adata_myo_full = utils.adata.load_h5ad("anndata_5_Myocardium_sub_clusters_annotated.h5ad")
# subset
adata_myo = adata_myo_full[adata_myo_full.obs['celltypes'].isin(['Atrium', 'Ventricle'])]

display(adata_myo)

In [ ]:
# Visualize cells in UMAP
sc.pl.umap(adata_myo_full, color = ['celltypes', 'sub_clusters'], title = [], legend_loc = 'right margin')

## Download receptor-ligand database
Provide a path to a table with receptor-ligand interactions. Will be stored in the adata object (`adata.uns['receptor-ligand]['database']`).

In [ ]:
adata_myo_full_sub = adata_myo_full[~adata_myo_full.obs['sub_clusters'].isin(['8'])]
adata_myo_full_sub.obs['sub_clusters'].unique()

In [ ]:
rl.download_db(adata=adata_myo_full,
               db_path='/mnt/flatfiles/receptor_ligand/zebrafish_rl_db_cellchat.txt',
               ligand_column='ligand',
               receptor_column='receptor',
               inplace=True,
               overwrite=True)

## Compute interactions
Create a table of receptor-ligand interactions between given groups. This table is used for plotting. Will be stored in the adata object (`adata.uns['receptor-ligand']['interactions']`)

### Clusters

In [ ]:
rl.calculate_interaction_table(adata=adata_myo_full,
                               cluster_column="sub_clusters",
                               gene_index=None,
                               normalize=1000,
                               inplace=True,
                               overwrite=False)

# Plotting

## Violin
Show the distribution of interaction scores for all group combinations.

Scores `> 0` can be interpreted as receptor-ligand interactions enriched for group combination.  
Scores `< 0` can be interpreted as receptor-ligand interactions depleted for group combination.

In [ ]:
rl.interaction_violin_plot(adata_myo_full,
                           min_perc=violin_min_perc,
                           save=f"rl_{violin_out_suffix}_myo.pdf",
                           figsize=violin_figsize,
                           dpi=dpi)

## Network
Show a network graph of number of interactions between groups.

In [ ]:
rl.hairball(adata_myo_full,
        min_perc=net_min_perc,
        interaction_score=net_interaction_score,
        interaction_perc=net_interaction_perc,
        save=f"rl_{net_out_suffix}_myo.pdf",
        title=net_title,
        color_min=net_color_min,
        color_max=net_color_max,
        restrict_to=net_restrict_to,
        show_count=net_show_count
       )

In [ ]:
_ = rl.cyclone(adata=adata_myo_full,
               min_perc=net_min_perc,
               interaction_score=net_interaction_score,
               interaction_perc=net_interaction_perc,
               save=f"rl_cyclone_{net_out_suffix}_myo_nogene.pdf",
               color_min=net_color_min,
               color_max=net_color_max,
               title=net_title,
               directional=net_directional,
               sector_size_is_cluster_size=net_sector_size_is_cluster_size,
               show_genes=False,
               gene_amount=net_gene_amount, dpi=300, colormap='Blues'
)

## Receptor-ligand connections
Show a detailed view on receptor-ligand pairs and their strength between groups.

- __receptor-/ ligand score__: Gene enriched (`> 0`) or depleted (`< 0`) for specific group.
- __receptor-/ ligand percent__: Percent of cells in a group expressing gene.
- __interaction score__: receptor-ligand pair enriched (`> 0`) or depleted (`< 0`) between groups.

In [ ]:
con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 0 & receptor_percent >= 40 & ligand_percent >= 40" # Conditions to filter the interaction table

rl.connectionPlot(adata=adata_myo_full, 
                  restrict_to=con_restrict_to,
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}_myo_subclusters.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")

In [ ]:
interaction_table = rl.get_interactions(adata_myo_full,
                                        min_perc = tab_min_perc,
                                        interaction_score = tab_interaction_score,
                                        interaction_perc = tab_interaction_perc,
                                        save = f"rl_{tab_out_suffix}_myo_subclusters.tsv").reset_index(drop=True)

In [ ]:
interaction_table

In [ ]:
# exclude intra-connections and connections with count < 500
myo_filtered = filter_cyclone(adata_myo_full, interaction_table, min_count=500)

In [ ]:
_ = rl.cyclone(adata=myo_filtered[myo_filtered.obs['sub_clusters'] != '8'],
               min_perc=net_min_perc,
               interaction_score=net_interaction_score,
               interaction_perc=net_interaction_perc,
               save=f"rl_cyclone_{net_out_suffix}_myo_nogene.pdf",
               color_min=500,
               color_max=net_color_max,
               title=net_title,
               directional=net_directional,
               sector_size_is_cluster_size=net_sector_size_is_cluster_size,
               show_genes=False,
               gene_amount=net_gene_amount, dpi=300, colormap='Blues'
)

In [ ]:
interaction_table = rl.get_interactions(myo_filtered,
                                        min_perc = tab_min_perc,
                                        interaction_score = tab_interaction_score,
                                        interaction_perc = tab_interaction_perc,
                                        save = f"rl_{tab_out_suffix}_myo_subclusters_filtered.tsv").reset_index(drop=True)

In [ ]:
interaction_table

# Atrium and Ventricle

In [ ]:
# Visualize cells in UMAP
sc.pl.umap(adata_myo, color = ['celltypes'], title = [], legend_loc = 'right margin')

## Download receptor-ligand database
Provide a path to a table with receptor-ligand interactions. Will be stored in the adata object (`adata.uns['receptor-ligand]['database']`).

In [ ]:
rl.download_db(adata=adata_myo,
           db_path=db_path,
           ligand_column='ligand',
           receptor_column='receptor',
           inplace=True,
           overwrite=True)

## Compute interactions
Create a table of receptor-ligand interactions between given groups. This table is used for plotting. Will be stored in the adata object (`adata.uns['receptor-ligand']['interactions']`)

In [ ]:
rl.calculate_interaction_table(adata=adata_myo,
                               cluster_column="celltypes",
                               gene_index=None,
                               normalize=1000,
                               inplace=True,
                               overwrite=False)

## Plotting

## Violin
Show the distribution of interaction scores for all group combinations.

Scores `> 0` can be interpreted as receptor-ligand interactions enriched for group combination.  
Scores `< 0` can be interpreted as receptor-ligand interactions depleted for group combination.

In [ ]:
rl.interaction_violin_plot(adata_myo,
                           min_perc=violin_min_perc,
                           save=f"rl_{violin_out_suffix}_myo_celltypes.pdf",
                           figsize=violin_figsize,
                           dpi=dpi)

## Network
Show a network graph of number of interactions between groups.

In [ ]:
rl.hairball(adata_myo,
            min_perc=net_min_perc,
            interaction_score=net_interaction_score,
            interaction_perc=net_interaction_perc,
            save=f"rl_{net_out_suffix}_myo_celltypes.pdf",
            title=net_title,
            color_min=net_color_min,
            color_max=net_color_max,
            restrict_to=net_restrict_to,
            show_count=net_show_count
           )

In [ ]:
_ = rl.cyclone(adata=adata_myo,
               min_perc=net_min_perc,
               interaction_score=net_interaction_score,
               interaction_perc=net_interaction_perc,
               save=f"rl_cyclone_{net_out_suffix}_myo_celltypes.pdf",
               color_min=net_color_min,
               color_max=net_color_max,
               title=net_title,
               directional=net_directional,
               sector_size_is_cluster_size=net_sector_size_is_cluster_size,
               show_genes=net_show_genes,
               gene_amount=10
)

## Receptor-ligand connections
Show a detailed view on receptor-ligand pairs and their strength between groups.

- __receptor-/ ligand score__: Gene enriched (`> 0`) or depleted (`< 0`) for specific group.
- __receptor-/ ligand percent__: Percent of cells in a group expressing gene.
- __interaction score__: receptor-ligand pair enriched (`> 0`) or depleted (`< 0`) between groups.

In [ ]:
con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 0 & receptor_percent >= 30 & ligand_percent >= 30" # Conditions to filter the interaction table

rl.connectionPlot(adata=adata_myo, 
                  restrict_to=con_restrict_to,
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}_myo_celltypes.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")

# Endocardium vs Atrium and Ventricle

### Concatenate

In [ ]:
adata = adata_myo.concatenate(adata_endo)
adata

In [ ]:
adata.obs.drop(['LISI_score_X_pca', 'LISI_score_X_umap', 'celltypes',
                'sub_clusters'], axis=1, inplace=True)

In [ ]:
tools.highly_variable.annot_HVG(adata, hvg_range=(1000, 5000))
sc.pp.pca(adata, svd_solver='arpack', use_highly_variable=True)

n_pcs = tools.dim_reduction.define_PC(adata)
tools.dim_reduction.subset_PCA(adata, start=1, n_pcs=n_pcs)

sc.pp.neighbors(adata, n_neighbors=15)
# batch correction
adata_tmp = tools.norm_correct.batch_correction(adata, "batch", "harmony", highly_variable=True)
# Embed cells into a subspace by UMAP
sc.tl.umap(adata, min_dist=0.1, spread=1.5, n_components=2)

In [ ]:
embedding = "umap"   #umap or tsne
clustering_method = "leiden" #leiden/louvain

pl.clustering.search_clustering_parameters(adata, ncols=4, method=clustering_method, 
                                    embedding=embedding)

In [ ]:
#Choose final resolution
clustering_column = "leiden_0.1"
adata.obs["clustering"] = utils.tables.rename_categories(adata.obs[clustering_column])

In [ ]:
clustering_col = "clustering"
# Identify markers per cluster (adjust group fraction and fold change to filter genes)
tools.marker_genes.run_rank_genes(adata, clustering_col,
                                 min_in_group_fraction=0.25,
                                 min_fold_change=0.5,
                                 max_out_group_fraction=0.8)

In [ ]:
# Marker list from Marco
userDB = '../marker_list.csv'
tools.celltype_annotation.run_scsa(adata, species=None, user_db=userDB, key='rank_genes_clustering')

In [ ]:
adata = adata[~adata.obs['SCSA_pred_celltype'].isin(['Nervous system - 1', 'Blood - Immune cells', 'Blood - red blood cells'])]
adata

In [ ]:
# Visualize cells in UMAP
sc.pl.umap(adata, color = ['SCSA_pred_celltype'], title = [], legend_loc = 'right margin')

In [ ]:
utils.adata.save_h5ad(adata, 'anndata_myo_endo_rl.h5ad')

## Download receptor-ligand database
Provide a path to a table with receptor-ligand interactions. Will be stored in the adata object (`adata.uns['receptor-ligand]['database']`).

In [ ]:
adata = utils.adata.load_h5ad('anndata_myo_endo_rl.h5ad')
adata

In [ ]:
rl.download_db(adata=adata,
               db_path=db_path,
               ligand_column='ligand',
               receptor_column='receptor',
               inplace=True,
               overwrite=True)

## Compute interactions
Create a table of receptor-ligand interactions between given groups. This table is used for plotting. Will be stored in the adata object (`adata.uns['receptor-ligand']['interactions']`)

### Clusters

In [ ]:
rl.calculate_interaction_table(adata=adata,
                               cluster_column="SCSA_pred_celltype",
                               gene_index=None,
                               normalize=1000,
                               inplace=True,
                               overwrite=True)

# Plotting

## Violin
Show the distribution of interaction scores for all group combinations.

Scores `> 0` can be interpreted as receptor-ligand interactions enriched for group combination.  
Scores `< 0` can be interpreted as receptor-ligand interactions depleted for group combination.

In [ ]:
rl.interaction_violin_plot(adata,
                           min_perc=violin_min_perc,
                           save=f"rl_{violin_out_suffix}_EndovsMyo.pdf",
                           figsize=violin_figsize,
                           dpi=dpi)

## Network
Show a network graph of number of interactions between groups.

In [ ]:
rl.hairball(adata,
            min_perc=net_min_perc,
            interaction_score=net_interaction_score,
            interaction_perc=net_interaction_perc,
            save=f"rl_{net_out_suffix}_EndovsMyo.pdf",
            title=net_title,
            color_min=net_color_min,
            color_max=net_color_max,
            restrict_to=net_restrict_to,
            show_count=net_show_count
           )

In [ ]:
_ = rl.cyclone(adata=adata,
               min_perc=net_min_perc,
               interaction_score=net_interaction_score,
               interaction_perc=net_interaction_perc,
               save=f"rl_cyclone_{net_out_suffix}_EndovsMyo.pdf",
               color_min=net_color_min,
               color_max=net_color_max,
               title=net_title,
               directional=net_directional,
               sector_size_is_cluster_size=net_sector_size_is_cluster_size,
               show_genes=net_show_genes,
               gene_amount=10, dpi=300
)

In [ ]:
_ = rl.cyclone(adata=adata,
               min_perc=net_min_perc,
               interaction_score=net_interaction_score,
               interaction_perc=net_interaction_perc,
               save=f"rl_cyclone_{net_out_suffix}_EndovsMyo_nogene.pdf",
               color_min=net_color_min,
               color_max=net_color_max,
               title=net_title,
               directional=net_directional,
               sector_size_is_cluster_size=net_sector_size_is_cluster_size,
               show_genes=False,
               gene_amount=10, dpi=300, colormap='Blues'
)

## Receptor-ligand connections
Show a detailed view on receptor-ligand pairs and their strength between groups.

- __receptor-/ ligand score__: Gene enriched (`> 0`) or depleted (`< 0`) for specific group.
- __receptor-/ ligand percent__: Percent of cells in a group expressing gene.
- __interaction score__: receptor-ligand pair enriched (`> 0`) or depleted (`< 0`) between groups.

### Endocardium vs Atrium

In [ ]:
con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 0 & receptor_percent >= 40 & ligand_percent >= 40" # Conditions to filter the interaction table

rl.connectionPlot(adata=adata, 
                  restrict_to=['Endocardium', 'Myocardium - Atrium'],
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}_EndovsAtrium.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")

In [ ]:
interaction_table = rl.get_interactions(adata,
                                        min_perc = tab_min_perc,
                                        interaction_score = tab_interaction_score,
                                        interaction_perc = tab_interaction_perc).reset_index(drop=True)
interaction_table

In [ ]:
# exclude intra-connections and connections with count < 500

myo_endo_filtered = filter_cyclone(adata, interaction_table, min_count=0)

In [ ]:
interaction_table = rl.get_interactions(myo_endo_filtered,
                                        min_perc = tab_min_perc,
                                        interaction_score = tab_interaction_score,
                                        interaction_perc = tab_interaction_perc,
                                        group_a = ['Endocardium'],
                                        group_b = ['Myocardium - Atrium'],
                                        save = f"rl_{tab_out_suffix}_EndovsAtrium_filtered.tsv"
                                       )

In [ ]:
interaction_table

In [ ]:
con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 0 & receptor_percent >= 5 & ligand_percent >= 5" # Conditions to filter the interaction table

rl.connectionPlot(adata=myo_endo_filtered, 
                  restrict_to=['Endocardium', 'Myocardium - Atrium'],
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}_EndovsAtrium.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")

### Endocardium vs Ventricle

In [ ]:
con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 10 & receptor_percent >= 5 & ligand_percent >= 5" # Conditions to filter the interaction table

rl.connectionPlot(adata=myo_endo_filtered, 
                  restrict_to=['Endocardium', 'Myocardium - Ventricle'],
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}_EndovsVentricle.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")

In [ ]:
interaction_table = rl.get_interactions(myo_endo_filtered,
                                        min_perc = tab_min_perc,
                                        interaction_score = tab_interaction_score,
                                        interaction_perc = tab_interaction_perc,
                                        group_a = ['Endocardium'],
                                        group_b = ['Myocardium - Ventricle'],
                                        save = f"rl_{tab_out_suffix}_EndovsVentricle_filtered.tsv")

In [ ]:
interaction_table

In [ ]:
con_filter = "ligand_gene == 'nrg1' | \
            ligand_gene == 'nrg2a' | \
            ligand_gene == 'nrg2b' | \
            ligand_gene == 'nrg3b' | \
              receptor_gene == 'erbb3a' | \
              receptor_gene == 'erbb3b' | \
              receptor_gene == 'erbb4b'"  # Conditions to filter the interaction table

con_figsize = (5, 3)

rl.connectionPlot(adata=myo_endo_filtered, 
                  restrict_to=['Endocardium', 'Myocardium - Ventricle'],
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}_EndovsVentricle_neuregulins.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")

In [ ]:
con_filter = "ligand_gene == 'nrg1' | \
            ligand_gene == 'nrg2a' | \
            ligand_gene == 'nrg2b' | \
              receptor_gene == 'erbb3a' | \
              receptor_gene == 'erbb3b' | \
              receptor_gene == 'erbb4b'"  # Conditions to filter the interaction table

con_figsize = (5, 3)

rl.connectionPlot(adata=myo_endo_filtered, 
                  restrict_to=['Endocardium', 'Myocardium - Atrium'],
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}_EndovsAtrium_neuregulins.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")